# Boiler Digital Twin - LSTM Model Training

This notebook trains an LSTM model to predict Boiler Steam Temperature based on control inputs.

**Physics Logic:** Higher valve position (TV_8329ZC) adds cooling spray → Lower temperature (TE_8332A)

In [1]:
# Import libraries
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import pickle
from tqdm.notebook import tqdm

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")

TensorFlow version: 2.20.0
GPU Available: False


## 1. Load Data

In [2]:
# Try to load actual data, fallback to synthetic if not available
try:
    print("Loading data.csv...")
    df = pd.read_csv('data.csv')
    
    # Select required columns
    required_cols = ['TV_8329ZC.AV_0#', 'PT_8313A.AV_0#', 'FT_8301.AV_0#', 'TE_8332A.AV_0#']
    df = df[required_cols].copy()
    
    # Rename for clarity
    df.columns = ['valve_position', 'pressure', 'flow_rate', 'temperature']
    
    print(f"✓ Loaded {len(df)} rows from data.csv")
    print(f"Columns: {list(df.columns)}")
    print(f"\nFirst few rows:")
    print(df.head())
    
except (FileNotFoundError, KeyError) as e:
    print(f"⚠ Could not load data.csv: {e}")
    print("Generating synthetic data for testing...")
    
    # Generate synthetic data with physics-based relationship
    n_samples = 10000
    t = np.linspace(0, 100, n_samples)
    
    # Valve position varies (main control variable)
    valve = 50 + 30 * np.sin(0.1 * t) + np.random.normal(0, 3, n_samples)
    
    # Context variables
    pressure = 101 + 5 * np.sin(0.05 * t) + np.random.normal(0, 0.5, n_samples)
    flow = 45 + 10 * np.sin(0.08 * t) + np.random.normal(0, 1, n_samples)
    
    # Temperature: INVERSE relationship with valve (physics-based)
    # Higher valve → more cooling → lower temp
    base_temp = 540
    temp_reduction = -0.5 * valve  # Negative coefficient for inverse relationship
    pressure_effect = 0.2 * pressure
    flow_effect = 0.1 * flow
    
    temperature = base_temp + temp_reduction + pressure_effect + flow_effect + np.random.normal(0, 2, n_samples)
    
    df = pd.DataFrame({
        'valve_position': valve,
        'pressure': pressure,
        'flow_rate': flow,
        'temperature': temperature
    })
    
    print(f"✓ Generated {len(df)} synthetic samples")
    print(df.head())

Loading data.csv...
✓ Loaded 86400 rows from data.csv
Columns: ['valve_position', 'pressure', 'flow_rate', 'temperature']

First few rows:
   valve_position  pressure  flow_rate  temperature
0           17.05   -142.28   47312.49       538.29
1           17.06   -142.79   47093.44       538.29
2           17.06   -135.23   47659.94       538.25
3           17.06   -147.23   47159.53       538.25
4           17.06   -147.91   46381.55       538.25


## 2. Preprocess Data

In [3]:
# Drop NaN values
print(f"Rows before dropping NaN: {len(df)}")
df = df.dropna()
print(f"Rows after dropping NaN: {len(df)}")

# Statistics
print(f"\nData statistics:")
print(df.describe())

Rows before dropping NaN: 86400
Rows after dropping NaN: 86400

Data statistics:
       valve_position      pressure     flow_rate   temperature
count    86400.000000  86400.000000  86400.000000  86400.000000
mean        16.082346   -137.711768  46462.882509    537.521130
std          4.857964     55.383279   1698.989047      4.328549
min          1.640000   -484.700000  40773.290000    517.050000
25%         12.540000   -171.240000  45286.340000    534.690000
50%         16.080000   -135.320000  46491.070000    537.710000
75%         18.990000   -100.400000  47610.840000    540.570000
max         35.480000     75.330000  53047.270000    550.490000


In [4]:
# Normalize features (0-1 range)
print("\nNormalizing features...")
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df.values)

print(f"✓ Scaled data shape: {scaled_data.shape}")
print(f"  Features: valve_position, pressure, flow_rate, temperature")

# Save scaler for later use
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print("✓ Saved scaler.pkl")


Normalizing features...
✓ Scaled data shape: (86400, 4)
  Features: valve_position, pressure, flow_rate, temperature
✓ Saved scaler.pkl


In [5]:
# Create sequences (60 timesteps to predict next step)
SEQUENCE_LENGTH = 60

def create_sequences(data, seq_len):
    X, y = [], []
    for i in tqdm(range(len(data) - seq_len), desc="Creating sequences"):
        # Input: 60 timesteps of [valve, pressure, flow, temp]
        X.append(data[i:i+seq_len])
        # Output: next temperature value (index 3)
        y.append(data[i+seq_len, 3])
    return np.array(X), np.array(y)

print(f"\nCreating sequences with length {SEQUENCE_LENGTH}...")
X, y = create_sequences(scaled_data, SEQUENCE_LENGTH)

print(f"✓ Created {len(X)} sequences")
print(f"  X shape: {X.shape} (samples, timesteps, features)")
print(f"  y shape: {y.shape} (samples,)")


Creating sequences with length 60...


Creating sequences:   0%|          | 0/86340 [00:00<?, ?it/s]

✓ Created 86340 sequences
  X shape: (86340, 60, 4) (samples, timesteps, features)
  y shape: (86340,) (samples,)


In [6]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nTrain/Test Split:")
print(f"  Training samples: {len(X_train)}")
print(f"  Testing samples: {len(X_test)}")


Train/Test Split:
  Training samples: 69072
  Testing samples: 17268


## 3. Build LSTM Model

In [7]:
# Build model
print("\nBuilding LSTM model...")

model = Sequential([
    LSTM(50, activation='relu', input_shape=(SEQUENCE_LENGTH, 4), return_sequences=False),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

print("✓ Model architecture:")
model.summary()


Building LSTM model...
✓ Model architecture:


c:\Users\Lenovo\Desktop\B.I.M.C.S\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 50)             │        11,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,051 (43.17 KB)

 Trainable params: 11,051 (43.17 KB)

 Non-trainable params: 0 (0.00 B)

## 4. Train Model

In [8]:
# Setup callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

print("\n" + "="*60)
print("Starting training...")
print("="*60)

# Train model
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[early_stop],
    verbose=1  # Show progress for each epoch
)

print("\n" + "="*60)
print("✓ Training completed!")
print("="*60)


Starting training...
Epoch 1/20
2159/2159 ━━━━━━━━━━━━━━━━━━━━ 29s 13ms/step - loss: 0.0047 - mae: 0.0485 - val_loss: 2.0676e-04 - val_mae: 0.0124
Epoch 2/20
2159/2159 ━━━━━━━━━━━━━━━━━━━━ 28s 13ms/step - loss: 8.2122e-04 - mae: 0.0210 - val_loss: 1.0637e-04 - val_mae: 0.0087
Epoch 3/20
2159/2159 ━━━━━━━━━━━━━━━━━━━━ 27s 13ms/step - loss: 3.9981e-04 - mae: 0.0144 - val_loss: 1.0500e-04 - val_mae: 0.0081
Epoch 4/20
2159/2159 ━━━━━━━━━━━━━━━━━━━━ 28s 13ms/step - loss: 3.6586e-04 - mae: 0.0135 - val_loss: 3.6140e-05 - val_mae: 0.0046
Epoch 5/20
2159/2159 ━━━━━━━━━━━━━━━━━━━━ 28s 13ms/step - loss: 3.4692e-04 - mae: 0.0130 - val_loss: 3.9963e-05 - val_mae: 0.0041
Epoch 6/20
2159/2159 ━━━━━━━━━━━━━━━━━━━━ 27s 13ms/step - loss: 3.3482e-04 - mae: 0.0127 - val_loss: 3.6075e-05 - val_mae: 0.0051
Epoch 7/20
2159/2159 ━━━━━━━━━━━━━━━━━━━━ 28s 13ms/step - loss: 3.3368e-04 - mae: 0.0125 - val_loss: 1.4024e-05 - val_mae: 0.0028
Epoch 8/20
2159/2159 ━━━━━━━━━━━━━━━━━━━━ 28s 13ms/step - loss: 3.2980e-

In [9]:
# Evaluate model
print("\nEvaluating model on test set...")
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)

print(f"\n📊 Model Performance:")
print(f"  Test Loss (MSE): {test_loss:.6f}")
print(f"  Test MAE: {test_mae:.6f}")

# Sample predictions
print("\nSample predictions (normalized):")
sample_preds = model.predict(X_test[:5], verbose=0)
for i in range(5):
    print(f"  Sample {i+1}: Predicted={sample_preds[i][0]:.4f}, Actual={y_test[i]:.4f}")


Evaluating model on test set...

📊 Model Performance:
  Test Loss (MSE): 0.000009
  Test MAE: 0.002371

Sample predictions (normalized):
  Sample 1: Predicted=0.5766, Actual=0.5786
  Sample 2: Predicted=0.7137, Actual=0.7162
  Sample 3: Predicted=0.5978, Actual=0.6044
  Sample 4: Predicted=0.5806, Actual=0.5757
  Sample 5: Predicted=0.5441, Actual=0.5437


## 5. Save Model

In [10]:
# Save model
model.save('boiler_model.keras')
print("\n✓ Model saved as 'boiler_model.keras'")
print("✓ Scaler saved as 'scaler.pkl'")

print("\n" + "="*60)
print("🎉 ALL DONE! Model training complete.")
print("="*60)
print("\nNext steps:")
print("  1. Use 'boiler_model.keras' for predictions")
print("  2. Use 'scaler.pkl' to normalize input data")
print("  3. Run 'python main.py' to start the FastAPI server")


✓ Model saved as 'boiler_model.keras'
✓ Scaler saved as 'scaler.pkl'

🎉 ALL DONE! Model training complete.

Next steps:
  1. Use 'boiler_model.keras' for predictions
  2. Use 'scaler.pkl' to normalize input data
  3. Run 'python main.py' to start the FastAPI server
